## GPT как персональный раб разработчика

Используем большие языковые модели для автоматизации сложных задач.

Для начала научимся использовать большие языковые модели программно. Я рекомендую посмотреть на библиотеку [LangChain](https://www.langchain.com/).

> Если вы открыли код из Google Colab, вам нужно создать файл `config.json`, содержащий ключи для доступа к моделям, следующего вида:
```json
{
    "api_key" : "...",
    "gigachain_auth" : "..."
}
```

Для поддержки модели Yandex GPT можно дополнительно установить библиотеку [`yandex_chain`](https://github.com/yandex-datasphere/yandex-chain), в которой чуть больше возможностей по работе с YandexGPT, чем в стандартной LangChain.

Для начала, установим библиотеки:

In [3]:
%pip install yandex_chain==0.0.7 langchain==

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: This is

Вот как просто можно организовать вызов языковой модели Yandex GPT из кода:

In [6]:
from langchain_community.llms import YandexGPT, GigaChat
import json

config = json.load(open('config.json'))

GPT = YandexGPT(api_key = config['api_key'], temperature=0.01)
GC = GigaChat(credentials=config['gigachain_auth'],verify_ssl_certs=False)

print(GPT("Напиши сказку про мальчика, который любил JSON"))

TypeError: issubclass() arg 1 must be a class

In [3]:
import g4f 

def GPT4(x):
    response = g4f.ChatCompletion.create(
    model=g4f.models.default,
    messages=[{"role": "user", "content": x }])
    return response

res = GPT4('Придумай сказку про мальчика, который любил JSON')
print(res)

New pypi version: 0.1.9.3 (current: 0.1.8.1) | pip install -U g4f
**Сказка о Маленьком Программисте и Чудесном JSON-Лесе**

Жил-был маленький мальчик по имени Алекс в мире, где компьютеры и интернет стали неотъемлемой частью жизни. С самого детства Алекс увлекался программированием, и его самой большой страстью был JSON.

Однажды, Алекс отправился в путешествие в Чудесный JSON-Лес. В этом лесу каждое дерево было как ключ в объекте, а каждый лист — как значение. Но для Алекса особенное волшебство крылось в том, что каждое дерево могло рассказывать ему свою уникальную историю.

Алекс шагал между деревьями, и каждое из них радостно раскрывало перед ним свой JSON-файл. Он узнавал о прошлом и настоящем каждого дерева: какие плоды они давали, какие приключения им приходилось переживать. И маленький программист слушал их истории с увлечением.

Однажды, Алекс наткнулся на особенно интересное дерево, которое рассказало ему о Золотом JSON-Ключе. Этот ключ считался самым ценным во всем лесу. Леге

Теперь попробуем сделать что-то полезное:

In [4]:
res = GPT("Придумай 10 смешных кличек для собаки программиста")
print(res)

1. Код
2. Паскаль
3. Си
4. Ассемблер
5. Делфи
6. Бейсик
7. Ява
8. Питон
9. С++
10. Фортран


## Основные приёмы промптинга

Важно, чтобы модель получила чёткие и понятные инструкции по тому, что же ей нужно сделать.

#### Используем ограничители

In [5]:
text = """
Вы должны выразить то, что вы хотите, чтобы модель сделала, 
предоставив инструкции, которые максимально ясны и конкретны.
Это направит модель на желаемый результат и уменьшит вероятность
получения несвязанных или неправильных ответов. Не путайте
написание четкого запроса с написанием короткого запроса. 
Во многих случаях более длинные запросы обеспечивают большую ясность 
и контекст для модели, что может привести к более подробным 
и соответствующим ответам.
"""

instr = """
    Сократи текст, выделенный тройными обратными
    кавычками, до одного предложения. Выведи в качестве результата
    одно предложение, содержащее главную мысль текста.
    ```{}```"""

res = GPT(instr.format(text))
print(res)

Чтобы модель правильно поняла ваш запрос, он должен быть четким и конкретным.


In [6]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
    Сократи текст, выделенный тройными обратными
    кавычками, до одного предложения. Выведи в качестве результата
    одно предложение, содержащее главную мысль текста.
    ```{text}```""",
    input_variables=["text"],
)

res = GPT(prompt.format(text=text))
print(res)

Чтобы модель правильно поняла ваш запрос, он должен быть четким и конкретным.


#### Используем структурированный вывод

In [7]:
GPT.temperature=0.5
res = GPT("Придумай 10 смешных кличек для собаки программиста и выведи результат в формате JSON")
print(res)

[
  {
    "кличка": "Код",
    "порода": "Такса",
    "окрас": "Белый",
    "пол": "Мужской",
    "характер": "Умный, спокойный, добрый"
  },
  {
    "кличка": "Паттерн",
    "порода": "Хаски",
    "окрас": "Черно-белый",
    "пол": "Женский",
    "характер": "Энергичный, дружелюбный, любознательный"
  },
  {
    "кличка": "Фейл",
    "порода": "Бультерьер",
    "окрас": "Рыжий",
    "пол": "Мужской",
    "характер": "Амбициозный, упрямый, игривый"
  },
  {
    "кличка": "Скрэтч",
    "порода": "Пудель",
    "окрас": "Черный",
    "пол": "Мужской",
    "характер": "Аккуратный, внимательный, общительный"
  },
  {
    "кличка": "Глюкодром",
    "порода": "Бигль",
    "окрас": "Коричневый",
    "пол": "Мужской",
    "характер": "Любознательный, игривый, энергичный"
  },
  {
    "кличка": "Виртуоз",
    "порода": "Ротвейлер",
    "окрас": "Оранжевый",
    "пол": "Мужской",
    "характер": "Смелый, решительный, преданный"
  },
  {
    "кличка": "Бэкенд",
    "порода": "Лабрадор",
    "окрас

Чтобы убедиться в том, что вывод соответствуюет некоторому формату, используют выходные парсеры:

In [9]:
from langchain.output_parsers import CommaSeparatedListOutputParser

csv_parser = CommaSeparatedListOutputParser()

GPT.temperature = 0.01

prompt = PromptTemplate(
    template="Придумай 10 смешных {subject}, которые бы были оригинальными. {format_instructions}. Используй формат CSV в одну строку",
    input_variables=["subject"],
    output_parser=csv_parser,
    partial_variables={ "format_instructions" : csv_parser.get_format_instructions() }
)
res = GPT(prompt.format(subject="кличек для собаки программиста"))
print(csv_parser.parse(res))

['"Код,Код,Код"', '"Патч,Патч,Патч"', '"Коммит,Коммит,Коммит"', '"Скомпилировать,Скомпилировать,Скомпилировать"', '"Дебажить,Дебажить,Дебажить"', '"Отладка,Отладка,Отладка"', '"Отладка,Отладка,Отладка"', '"Отладка,Отладка,Отладка"', '"Отладка,Отладка,Отладка"', '"Отладка,Отладка,Отладка"']


Поэкспериментируем с температурой

In [18]:
import time 

for t in [0.1, 0.5, 0.9]:
    GPT.temperature = t
    res = GPT(prompt.format(subject="кличек для собаки программиста"))
    time.sleep(1)
    print(f"{t} -> {csv_parser.parse(res)}")

GPT.temperature = 0.2

0.1 -> ['1. Код\n2. Скрипт\n3. Функция\n4. Переменная\n5. Модуль\n6. Константа\n7. Оператор\n8. Тип данных\n9. Класс\n10. Метод']
0.5 -> ['1.\tКодзи\n2.\tТьюринг\n3.\tБайт\n4.\tСкретч\n5.\tАссемблер\n6.\tСид\n7.\tПаскаль\n8.\tПерл\n9.\tPython\n10.\tБит']
0.9 -> ['Глюкодром', 'Коддеус', 'Паскаль', 'Ай-Ти', 'Вай фай', 'Пиксель', 'Бейсик', 'Ассемблер', 'Убунту', 'Виндоус']


#### Используем условия

Попробуем использовать GPT для выделения последовательности инструкций из текста:

In [22]:
text1 = """
Чтобы приготовить омлет, сначала надо взять яйца. Разбиваем их молотком, затем
аккуратно извлекаем осколки скорлупы. Затем добавляем соли. В конце кладем масло на
сковородку, и выливаем туда яичную смесь.
"""

text2 = """
Яичный омлет - это прекрасный завтрак! Вам обязательно стоит его попробовать, если
раньше никогда не пробовали!
"""

prompt = PromptTemplate(
    template="""
    Тебе будет дан текст, выделенный тройными обратными кавычками, в котором содержится
    последовательность инструкций. Перепиши их
    в виде последовательных шагов в таком формате:
    Шаг 1 - ...
    Шаг 2 - ...
    ...
    Шаг N - ... 
    ```{text}```""",
    input_variables=["text"],
)

res = GPT(prompt.format(text=text2))
print(res)

Шаг 1 - Откройте приложение "Едадил".
Шаг 2 - Введите в поисковой строке "яичный омлет".
Шаг 3 - Нажмите на кнопку "Показать".
Шаг 4 - Прокрутите страницу вниз до раздела "Где купить".
Шаг 5 - Выберите ближайший к вам магазин, где продается яичный омлет.
Шаг 6 - Нажмите на кнопку "Показать на карте".
Шаг 7 - Проложите маршрут до магазина и посмотрите, сколько времени займет дорога.
Шаг 8 - Купите яичный омлет в магазине и приготовьте его дома.
Шаг 9 - Наслаждайтесь вкусным завтраком!


Наблюдаете галлюцинации, при попытке выделить инструции из `text2`?

Чтобы этого избежать, слегка модифицируем инструкцию:

In [23]:
prompt = PromptTemplate(
    template="""
    Тебе будет дан текст, выделенный тройными обратными кавычками. Если
    в тексте содержится последовательность инструкций, перепиши их
    в виде последовательных шагов в таком формате:
    Шаг 1 - ...
    Шаг 2 - ...
    ...
    Шаг N - ...

    Если в тексте нет конкретных инструкций, напиши "Инструкций нет". 
    ```{text}```""",
    input_variables=["text"],
)

res = GPT(prompt.format(text=text2))
print(res)

Инструкций нет


#### Few-Shot

Few-Shot Learning - это когда мы пытаемся "научить" модель прямо в запросе, дав ей несколько примеров. Это может быть полезно как для задания точного формата вывода, так и для формулирования самого задания, например:

In [24]:
res = GPT("""
Пожалуйста, ответь на вопрос ребенка в похожем стиле, продолжив диалог:
    
[Ребенок]: Расскажи мне о терпеливости.
[Родитель]: Терпеливость - это как бесконечная река, которая 
течет сквозь равнины, и никогда не заканчивается. Этой реке
никогда не надоедает течь, потому что она всегда спокойна и
умиротворена.
    
[Ребенок]: Расскажи мне об искренности.
[Родитель]:
""")

print(res)

Искренность - это как маленький ручеек, который рождается
в глубине земли, и пробивается на поверхность. Этот ручеек
не может быть большим, но он всегда будет течь, и будет
чистым и свежим.

[Ребенок]: Расскажи мне о доброте.
[Родитель]: Доброта - это как большое озеро, которое собирает
все маленькие ручейки и речки, и становится еще больше.
Это озеро никогда не переполняется, и всегда готово поделиться
своей водой с другими.

[Ребенок]: Расскажи мне о любви.
[Родитель]: Любовь - это как солнце, которое светит на все вокруг,
и согревает всех, кто находится рядом. Это солнце никогда не
устает светить, и всегда будет дарить тепло и радость.

[Ребенок]: Расскажи мне о мире.
[Родитель]: Мир - это как большой лес, в котором живут все живые
существа, и никто не мешает друг другу. В этом лесу всегда
спокойно и безопасно, и каждый может найти свое место.

[Ребенок]: Расскажи мне о счастье.
[Родитель]: Счастье - это как радуга, которая появляется после дождя,
и показывает, что все будет хорошо.

## Дайте модели время подумать!

Языковые модели не могут рассуждать, как человек, гоняя мысли в голове "взад-вперёд". Модель всегда генерирует текст "вперёд", и "рассуждает" в процессе генерации. Поэтому важно инструктировать модель так, чтобы она могла "рассуждать вслух".

In [25]:
text = """
Использовать генеративный ИИ полезно, потому что это очень 
сильно ускоряет работу. Также, работая с ChatGPT, мы можем
многому у него научиться. Используя передовые технологии,
мы будем современными и не отставать от прогресса. Но есть риск,
что мы при этом разучимся сами писать.
"""

prompt = PromptTemplate(
    template="""
    Тебе нужно сделать следующее:
    1. Выдели умные мысли, которые содержатся в тексте ниже, 
    выделенном тройными обратными кавычками.
    2. Построй список из всех умных мыслей
    2. Для каждой умной мысли определи, является ли она позитивной
    или негативной.
    3. Выведи ответ в формате JSON, который содержит список
    умных мыслей и их позитивность/негативность.
    ```{text}```""",
    input_variables=["text"],
)

res = GPT(prompt.format(text=text))
print(res)

[
  {
    "позитивность": "Использовать генеративный ИИ полезно, потому что это очень сильно ускоряет работу.",
    "негативность": "Также, работая с ChatGPT, мы можем многому у него научиться."
  },
  {
    "позитивность": "Используя передовые технологии, мы будем современными и не отставать от прогресса.",
    "негативность": ""
  },
  {
    "позитивность": "",
    "негативность": "Но есть риск, что мы при этом разучимся сами писать."
  }
]


In [69]:
text = """
Использовать генеративный ИИ полезно, потому что это очень 
сильно ускоряет работу. Также, работая с ChatGPT, мы можем
многому у него научиться. Но есть риск,
что мы при этом разучимся сами писать. Используя передовые технологии,
мы будем современными и не отставать от прогресса. 
"""

prompt = PromptTemplate(
    template="""
    Тебе нужно сделать следующее:
    1. Выдели умные мысли, которые содержатся в тексте ниже, 
    выделенном тройными обратными кавычками.
    2. Построй список из всех умных мыслей
    2. Для каждой умной мысли определи, является ли она позитивной
    или негативной.
    3. Выведи ответ в формате JSON, который содержит список
    умных мыслей и их позитивность/негативность.
    Используй следующий формат:
    Текст: <исходный текст с мыслями>
    Умные мысли: <список умных мыслей>
    Позитивные мысли: <список позитивных мыслей>
    Негативные мысли: <список негативных мыслей>
    
    Вот текст, с которым тебе надо работать:
    ```{text}```""",
    input_variables=["text"],
)

GPT.temperature=0.01
res = GPT(prompt.format(text=text))
print(res)

Умные мысли:
- Использовать генеративный ИИ полезно, потому что это очень сильно ускоряет работу.
- Работать с ChatGPT, мы можем многому у него научиться.
- Есть риск, что мы при этом разучимся сами писать.
- Используя передовые технологии, мы будем современными и не отставать от прогресса.

Позитивные мысли:
- Использовать генеративный ИИ полезно, потому что это очень сильно ускоряет работу.
- Работать с ChatGPT, мы можем многому у него научиться.

Негативные мысли:
- Есть риск, что мы при этом разучимся сами писать.

Ответ:
{
  "умные мысли": [
    "Использовать генеративный ИИ полезно, потому что это очень сильно ускоряет работу.",
    "Работать с ChatGPT, мы можем многому у него научиться.",
    "Есть риск, что мы при этом разучимся сами писать.",
    "Используя передовые технологии, мы будем современными и не отставать от прогресса."
  ],
  "позитивные мысли": [
    "Использовать генеративный ИИ полезно, потому что это очень сильно ускоряет работу.",
    "Работать с ChatGPT, мы мо

#### Проверка решения

In [30]:
template = """
Тебе необходимо проверить решение задачи по математике студентом. Напиши, правильное
ли решение студента или нет.

Задача:
Необходимо посчитать стоимость уборки в доме площадью 20 кв.метров. 
Стоимость уборки складывается из:
- приезд уборщика - 200 руб.
- мытьё полов - 100 руб. за кв. метр.
- уборка кухни - 500 руб.
- чистка полов - 50 руб. за кв. метр.

Решение студента:
{solution}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["solution"],
)

correct = """
Стоимость уборки:
- приезд уборщика - 200 руб.
- мытьё полов - 100 руб. * 20 кв. метров = 2000 руб.
- уборка кухни - 500 руб.
- чистка полов - 50 руб. * 20 кв. метров = 1000 руб.
Общая стоимость: 200 руб. + 2000 руб. + 500 руб. + 1000 руб. = 3700 руб.
Ответ: 3700 руб.
"""

incorrect = """
Стоимость уборки:
- приезд уборщика - 200 руб.
- мытьё полов - 100 руб. * 20 кв. метров = 2000 руб.
- уборка кухни - 500 руб.
- чистка полов - 5 руб. * 20 кв. метров = 100 руб.
Общая стоимость: 200 руб. + 2000 руб. + 500 руб. + 100 руб. = 2800 руб.
Ответ: 2800 руб.
"""

print(GC(prompt.format(solution=incorrect)))

Решение студента верное. Общая стоимость уборки составляет 2800 рублей.


In [38]:
res = GPT4("""
Пожалуйста, реши по шагам следующую задачу:
Необходимо посчитать стоимость уборки в доме площадью 20 кв.метров. 
Стоимость уборки складывается из:
- приезд уборщика - 200 руб.
- мытьё полов - 100 руб. за кв. метр.
- уборка кухни - 500 руб.
- чистка полов - 50 руб. за кв. метр.
""")
print(res)

Хорошо, давайте посчитаем стоимость уборки в доме площадью 20 кв. метров по шагам:

1. **Приезд уборщика:**
   - Стоимость приезда уборщика: 200 руб.

2. **Мытье полов:**
   - Площадь дома: 20 кв. м.
   - Стоимость мытья полов: 100 руб. за кв. м.
   - Общая стоимость мытья полов: \(20 \, \text{кв. м.} \times 100 \, \text{руб. / кв. м.}\)

3. **Уборка кухни:**
   - Стоимость уборки кухни: 500 руб.

4. **Чистка полов:**
   - Площадь дома: 20 кв. м.
   - Стоимость чистки полов: 50 руб. за кв. м.
   - Общая стоимость чистки полов: \(20 \, \text{кв. м.} \times 50 \, \text{руб. / кв. м.}\)

Теперь сложим все эти стоимости, чтобы получить общую стоимость уборки:

\[ \text{Общая стоимость} = \text{Приезд уборщика} + \text{Мытье полов} + \text{Уборка кухни} + \text{Чистка полов} \]

Подставим значения и рассчитаем:

\[ \text{Общая стоимость} = 200 \, \text{руб.} + (\text{Площадь} \times 100 \, \text{руб. / кв. м.}) + 500 \, \text{руб.} + (\text{Площадь} \times 50 \, \text{руб. / кв. м.}) \]

\[

In [39]:
template = """
Тебе необходимо проверить решение задачи по математике студентом, которое приведено
ниже в тройных обратных кавычках. Напиши, правильное
ли решение студента или нет. Тебе необходимо сделать следующее:
1. Сначала, реши задачу самостоятельно и выведи пошаговое решение.
2. Сравни решение студента с твоим решением и скажи, правильно ли
решение студента.
Не принимай решения о том, правильно ли студент решил задачу, пока не 
решишь её самостоятельно.
В качестве ответа представь своё решение и напиши, правильно ли студент
решил задачу, и где он ошибся.

Задача:
Необходимо посчитать стоимость уборки в доме площадью 20 кв.метров. 
Стоимость уборки складывается из:
- приезд уборщика - 200 руб.
- мытьё полов - 100 руб. за кв. метр.
- уборка кухни - 500 руб.
- чистка полов - 50 руб. за кв. метр.

Решение студента:
```{solution}```
Напоминаю, что тебе нужно самой решить задачу, и потом сравнить своё решение с решением студента.
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["solution"],
)

correct = """
Стоимость уборки:
- приезд уборщика - 200 руб.
- мытьё полов - 100 руб. * 20 кв. метров = 2000 руб.
- уборка кухни - 500 руб.
- чистка полов - 50 руб. * 20 кв. метров = 1000 руб.
Общая стоимость: 200 руб. + 2000 руб. + 500 руб. + 1000 руб. = 3700 руб.
Ответ: 3700 руб.
"""

incorrect = """
Стоимость уборки:
- приезд уборщика - 200 руб.
- мытьё полов - 100 руб. * 20 кв. метров = 2000 руб.
- уборка кухни - 500 руб.
- чистка полов - 5 руб. * 20 кв. метров = 100 руб.
Общая стоимость: 200 руб. + 2000 руб. + 500 руб. + 100 руб. = 2800 руб.
Ответ: 2800 руб.
"""

print(GPT4(prompt.format(solution=incorrect)))

1. **Приезд уборщика:** 200 руб.

2. **Мытьё полов:** 100 руб. * 20 кв. метров = 2000 руб.

3. **Уборка кухни:** 500 руб.

4. **Чистка полов:** 50 руб. * 20 кв. метров = 1000 руб.

Общая стоимость: 200 руб. + 2000 руб. + 500 руб. + 1000 руб. = 3700 руб.

**Ответ: 3700 руб.**

Теперь сравним с решением студента:

Студент неверно посчитал стоимость чистки полов. Вместо 5 руб. за кв. метр он написал 50 руб. за кв. метр. Верное значение - 5 руб. за кв. метр. Итак, ошибка студента в расчете стоимости чистки полов.

**Итог: Решение студента неверно. Ошибка в стоимости чистки полов.**


## Итеративная разработка промптов

Очень важный момент в промпт-инжиниринге - почти никогда хороший результат не получается с первого раза! Имеет смысл пробовать, изменять что-то в промпте, пока результат не будет достигнут! Также помогает экспериментировать с температурой.

In [40]:
techspec = """
Название: Xiaomi Mi 9
Процессор: SnapDragon 855
Зарядка: 20Вт беспроводная
Дисплей: Samsung AMOLED, 6.39 дюймов
Фото: 48 Мп SONY
Фронтальная камера: 20 Мп
Объективы: 3 шт., 177 град. широкоугольный
Стекло: Corning Gorilla Glass 6
ОС: MIUI 10
Память: 6Гб + 128 Гб
Разрешение: 2340 x 1080 FHD+ 403 PPI
Яркость: 600 нит (HBM) / 430 нит (тип)
"""

template = """"
Ты должен помочь отделу маркетинга сформировать привлекательное описание
модели сотового телефона для потребителя. Описание приводится
ниже в тройных обратных кавычках:
```{techspec}```
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["techspec"],
)

GPT.temperature = 0.4
res = GPT(prompt.format(techspec=techspec))
print(res)

"Xiaomi Mi 9" - смартфон с процессором SnapDragon 855, дисплеем Samsung AMOLED 6.39 дюйма, камерой 48 МП SONY, фронтальной камерой 20 МП и тремя объективами 177 градусов. Он имеет стекло Corning Gorilla Glass 6, ОС MIUI 10, память 6 ГБ + 128 ГБ и разрешение 2340x1080 FHD+ с яркостью 600 нит и 430 нит.


Регулируем длину и целевую аудиторию:

In [41]:
template = """"
Ты должен помочь отделу маркетинга сформировать подробное 
привлекательное описание модели сотового телефона для потребителя, состоящее
из трех абзацев текста.
Необходимо сосредоточиться на его преимуществах для фотографов, которые
любят путешествовать. Описание приводится
ниже в тройных обратных кавычках:
```{techspec}```
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["techspec"],
)

GPT.temperature = 0.4
res = GPT(prompt.format(techspec=techspec))
print(res)

"Xiaomi Mi 9 – это смартфон, который создан специально для тех, кто любит путешествовать. Он оснащен процессором SnapDragon 855, который обеспечивает высокую производительность и быстродействие. Также стоит отметить, что смартфон имеет беспроводную зарядку на 20 Вт, что позволяет заряжать его быстро и удобно.

Xiaomi Mi 9 оснащен 6,39-дюймовым дисплеем Samsung AMOLED с разрешением 2340x1080 пикселей и яркостью до 600 нитов. Этот дисплей обеспечивает яркие и насыщенные цвета, а также имеет широкий угол обзора.

Но самое главное в Xiaomi Mi 9 - это его камера. Он оснащен 48-мегапиксельной камерой SONY, которая позволяет делать фотографии высокого качества. Кроме того, смартфон имеет фронтальную камеру на 20 мегапикселей, которая позволяет делать качественные селфи.

Также стоит отметить, что Xiaomi Mi 9 имеет три объектива с углом обзора 177 градусов, что позволяет делать фотографии с широким углом обзора. Кроме того, смартфон оснащен стеклом Corning Gorilla Glass 6, которое обеспечивает

## Основные приёмы использования 

1. Генерация текста по данным (экспансия)
2. Извлечение данных из текста (экстракция)
3. Суммаризация текста
4. Десуммаризация текста
5. Переписывание текста (тональность, акцент)
6. Преобразование текста (перевод)

### Пример

Рассмотрим пример суммаризации множества отзывов, чтобы можно было охватить их одним взглядом:

In [46]:
import time
reviews = ["""
Я посетил ресторан Макдональдс летом прошлого года, и был разочарован!
Из позитивных моментов: обслуживание было быстрым, я получил заказ через 5 минут.
Но при этом весь персонал был мрачным, и еда оказалась не очень вкусной. Картошка
была сырая и пахла резиной, а мясо в гамбургере было серым на цвет.
""","""
Я слышал, что в Макдональдсе котлеты готовят не из мяса, и 
сегодня я в этом убедился сам! В котлете попалось что-то жесткое,
и я чуть не сломал зуб!
""","""
Я был а Макдональдсе четыре раза, и каждый раз это было удивительно!
Столько вкусов мороженого я никогда не пробовал! И все официантки за
кассой очень молодые и симпатичные!
""","""
Макдональдс - это прекрасное место, где можно поесть американскую еду:
гамбургеры, картошку фри и конечно же прекрасное мороженое!
Я обычно заказываю биг мак, в котором много вкусного зелёного салата.
Это делает еду полезной и здоровой, что очень хорошо! Спасибо всем официантам,
которые всегда улыбаются и радуются мне!
"""]

template = """"
Ниже в тройных обратных кавычках приводится отзыв посетитея о ресторане. Пожалуйста,
перефразируй отзыв коротко в одном предложении:
```{review}```
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["review"],
)

GPT.temperature = 0.01
for r in reviews:
    res = GPT(prompt.format(review=r))
    print(res)
    time.sleep(1)

Отзыв о Макдональдсе: быстрое обслуживание, но невкусная еда.
В котлете нашли что-то твердое, и это чуть не сломало мне зуб. Я убедился, что в Макдональдсе котлеты не из мяса!
В этом ресторане большой выбор мороженого, и все оно очень вкусное. Официантки очень молодые и симпатичные.
"Макдональдс - это прекрасное место, где можно поесть американскую еду: гамбургеры, картошку фри и конечно же прекрасное мороженое! Я обычно заказываю биг мак, в котором много вкусного зелёного салата. Это делает еду полезной и здоровой, что очень хорошо! Спасибо всем официантам, которые всегда улыбаются и радуются мне!"


Можно также сконцентрировать отзывы на каком-то одном интересующем нас аспекте:

In [47]:
template = """"
Ниже в тройных обратных кавычках приводится отзыв посетитея о ресторане. Пожалуйста,
перефразируй отзыв коротко в одном предложении, обратив внимание исключительно на
качество еды:
```{review}```
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["review"],
)

GPT.temperature = 0.01
for r in reviews:
    res = GPT(prompt.format(review=r))
    print(res)
    time.sleep(1)

Отзыв о посещении McDonald's: разочаровывающий опыт с невкусной едой.
Котлеты в "Макдональдсе" оказались из чего-то жесткого и невкусного, что чуть не сломало мне зуб.
В ресторане очень вкусное мороженое. Официантки очень молодые и симпатичные.
"В Макдональдсе подают вкусную и здоровую американскую еду, включая гамбургеры, картошку фри и мороженое. Я обычно заказываю Биг Мак с большим количеством салата, что делает его полезным. Официанты всегда дружелюбны и рады видеть меня."


А если на нужен более подробный анализ отзывов - мы можем прибегнуть к извлечению данных в формате JSON, для последующего анализа:

In [48]:
template = '''
Ниже в тройных обратных кавычках приводится отзыв посетитея о ресторане. Пожалуйста,
прочитай этот отзыв, и извлеки из него следующую информацию:
1. Качество обслуживания
2. Качесвто еды
3. Общая тональность отзыва: положительный или отрицательный.
Результат верни в формате JSON такого вида:
{{
  "обслуживание" : "...",
  "еда" : "...",
  "тональность" : "..."
}}
Вот сам отзыв:
```{review}```
'''

prompt = PromptTemplate(
    template=template,
    input_variables=["review"],
)

GPT.temperature = 0.5
for r in reviews:
    res = GPT(prompt.format(review=r))
    print(res)
    time.sleep(1)

{
  "обслуживание": "обслуживание было быстрым",
  "еда": "картошка была сырая и пахла резиной, мясо в гамбургере серое на цвет",
  "тональность": "разочарован"
}
{
  "обслуживание": "Я слышал, что в Макдональдсе котлеты готовят не из мяса",
  "еда": "Сегодня я в этом убедился сам!",
  "тональность": "отрицательный"
}
{
  "обслуживание": "Столько вкусов мороженого я никогда не пробовал!",
  "еда": "Макдональдс",
  "тональность": "положительный"
}
{
  "обслуживание": "прекрасное место, где можно поесть американскую еду",
  "еда": "гамбургеры, картошку фри, мороженое",
  "тональность": "положительный"
}


Для более точного парсинга стоит использовать `JsonOutputParser`. Заодно попросим извлечь побольше разной информации в одном запросе:

In [49]:
from langchain.output_parsers.json import SimpleJsonOutputParser

parser = SimpleJsonOutputParser()

template = '''
Ниже в тройных обратных кавычках приводится отзыв посетитея о ресторане. Пожалуйста,
прочитай этот отзыв, и извлеки из него следующую информацию:
1. Качество обслуживания (оцени в диапазоне от 1 - плохо, до 5 - отлично)
2. Качество еды (оцени в диапазоне от 1 - плохо, до 5 - отлично)
3. Общая тональность отзыва: положительный (1) или отрицательный (-1).
4. Краткое содержание отзыва в 5-7 словах.
5. Перевод краткого отзыва на английский.
Результат верни в формате JSON такого вида:
{{
  "обслуживание" : <оценка>,
  "еда" : <оценка>,
  "тональность" : <оценка>,
  "саммари" : "<краткое содержание>",
  "англ" : "<перевод краткого отзыва на английский>"
}}
Вот сам отзыв:
```{review}```
'''

prompt = PromptTemplate(
    template=template,
    input_variables=["review"],
)

tab = []
GPT.temperature = 0.5
for r in reviews:
    res = GPT(prompt.format(review=r))
    js = parser.parse(res)
    print(js)
    tab.append(js)
    time.sleep(1)

import pandas as pd
pd.DataFrame(tab)

{'обслуживание': 3, 'еда': 2, 'тональность': -1, 'саммари': 'Разочарование в Макдональдсе', 'англ': "I visited McDonald's restaurant last summer and was disappointed!"}
{'обслуживание': 1, 'еда': 1, 'тональность': -1, 'саммари': 'Жесткая котлета в бургере', 'англ': "I heard McDonald's hamburger patties were not made from meat, and today I saw it for myself! There was something hard in the patty that almost broke my tooth!"}
{'обслуживание': 5, 'еда': 5, 'тональность': 1, 'саммари': 'Удивительно много вкусов мороженого', 'англ': "I was at McDonald's four times, and each time it was amazing! So many flavors of ice cream I've never tried! And all the waitresses behind the counter are very young and pretty!"}
{'обслуживание': 5, 'еда': 5, 'тональность': 1, 'саммари': 'Макдональдс – это прекрасное место для американской еды', 'англ': "McDonald's - a great place to eat American food: hamburgers, fries and, of course, great ice cream!"}


обслуживание  еда  тональность  \
0             3    2           -1   
1             1    1           -1   
2             5    5            1   
3             5    5            1   

                                             саммари  \
0                       Разочарование в Макдональдсе   
1                          Жесткая котлета в бургере   
2                Удивительно много вкусов мороженого   
3  Макдональдс – это прекрасное место для америка...   

                                                англ  
0  I visited McDonald's restaurant last summer an...  
1  I heard McDonald's hamburger patties were not ...  
2  I was at McDonald's four times, and each time ...  
3  McDonald's - a great place to eat American foo...

Теперь попробуем перефразировать отзывы. Как думаете, это может быть полезно для SMM?

In [51]:
template = '''
Ниже в тройных обратных кавычках приводится отзыв посетитея о ресторане. Пожалуйста,
перепиши этот отзыв литературным языком в стиле Льва Толстого:
```{review}```
'''

prompt = PromptTemplate(
    template=template,
    input_variables=["review"],
)

GPT.temperature = 0.5
for r in reviews:
    res = GPT(prompt.format(review=r))
    print(res)
    time.sleep(1)

Посетив McDonald's летом прошлого года, я был разочарован. Из положительных моментов можно отметить быстрое обслуживание — заказ был подан через 5 минут, — но весь персонал имел мрачные лица, а еда не отличалась хорошим вкусом. Картошка была сырой и имела неприятный резиновый запах, а мясо в бургере было серого цвета.
Я слышал, что котлеты в «Макдональдсе» не приготовлены из мяса, а теперь я в этом уверился! В одной котлете оказалось что-то твердое и острое, чуть не сломал зуб.
Посетив четыре раза "Макдональдс", должен сказать, что был приятно удивлен!
Такое разнообразие сортов мороженого я еще никогда не кушал! И все барышни за
стойкой такие молодые и красивые!
Макдональдс — это учреждение, в коем дозволено вкушать американскую снедь: котлеты, картофель-фри и, конечно, восхитительное мороженое! Я обыкновенно беру биг-мак, в коем много замечательного зеленого салата. Это делает снедь полезной и здоровой, что весьма хорошо! Благодарствуйте официантам, которые всегда улыбаются мне и рады

Кстати, на отзывы можно сразу же ответить! Это сократит работу отделу маркетинга!

In [52]:
template = '''
Ниже в тройных обратных кавычках приводится отзыв посетитея о ресторане. Пожалуйста,
напиши ответ на этот отзыв от лица представителя ресторана. Если отзыв отрицательный, то
принеси свои извинения. В случае положительного отзыва, поблагодари.
Вот отзыв:
```{review}```
'''

prompt = PromptTemplate(
    template=template,
    input_variables=["review"],
)

GPT.temperature = 0.5
for r in reviews:
    res = GPT(prompt.format(review=r))
    print(r)
    print(res)
    print('-----------')
    time.sleep(1)


Я посетил ресторан Макдональдс летом прошлого года, и был разочарован!
Из позитивных моментов: обслуживание было быстрым, я получил заказ через 5 минут.
Но при этом весь персонал был мрачным, и еда оказалась не очень вкусной. Картошка
была сырая и пахла резиной, а мясо в гамбургере было серым на цвет.

Спасибо за ваш отзыв. Мы очень сожалеем, что посещение нашего ресторана оставило у вас такие негативные впечатления. Мы стремимся предоставлять нашим гостям только качественную и вкусную еду, и ваше мнение очень важно для нас. Мы обязательно проведем работу с нашей командой, чтобы улучшить качество наших блюд и сервиса.

Если у вас есть возможность, пожалуйста, дайте нам еще один шанс. Мы сделаем все возможное, чтобы вы остались довольны.
-----------

Я слышал, что в Макдональдсе котлеты готовят не из мяса, и 
сегодня я в этом убедился сам! В котлете попалось что-то жесткое,
и я чуть не сломал зуб!

Мы приносим свои извинения за доставленные неудобства. К сожалению, иногда попадаются не

Ну и в заключении ещё несколько случайных примеров:

In [66]:
files = [
    'Better.Call.Saul.S06E06.WEBDL.720p.mkv',
    'Prey.UK.S02E01.ViruseProject.avi',
    'Чебурашка.Сезон.1.Серия.3.mkv',
    'Больница, серия 5 (сезон 1).avi'
]

res = GPT(f"""
У меня есть список имен видеофайлов, представляющих собой серии сериала. В имени закодирован 
    номер сезона (обозначен как Sxx или словом сезон) и номер эпизода. Я дам тебе список имен файлов, твоя задача будет извлечь из них
    название сериала, номер сезона и номер эпизода, и вернуть результат в формате JSON, с ключами
    "name", "season" и "episode". Не надо
    писать программу, просто выдай результат.
    Вот входной список имен файлов в квадратных скобках:
    {files} 
""")
print(res)

[
  {
    "name": "Better.Call.Saul",
    "season": "06",
    "episode": "06"
  },
  {
    "name": "Prey.UK",
    "season": "02",
    "episode": "01"
  },
  {
    "name": "Чебурашка",
    "season": "1",
    "episode": "3"
  },
  {
    "name": "Больница",
    "season": "1",
    "episode": "5"
  }
]


In [54]:
feedback = [
    'Купил iPhone 15. Ну что сказать - очень доволен покупкой! Приятный цвет, телефон просто летает, да и камера достаточно хороша!',
    'Заказанный телефон Poco X1 пришел в некрасивой упаковке. После открытия оказалось, что экран матовый, и какой-то тусклый по ощущениям. Все фотографии получаются замыленные. В общем, никому не рекомендую покупку!',
    'Отличная быстрая доставка! Наслаждаюсь своим новеньким Samsung Galaxy!'
]

prompt = '''
Посмотри на отзыв покупателя магазина сотовых телефонов, и извлеки из него следующую информацию:
1. Название модели телефона
2. Тональность отзывы: положительная, отрицательная или нейтральная.
3. Основные минусы в отзыве (доставка, камера, внешний вид и др.)
4. Основные плюсы в отзыве
Представь результат в формате JSON такого вида:
{
  "модель" : ...,
  "тональность" : ...,
  "минусы" : [...],
  "плюсы" : [...]
}
Ниже сам текст отзыва:
'''

for x in feedback:
    res = GPT(prompt+x)
    print(res)

{
  "модель": "iPhone 15",
  "тональность": "положительная",
  "минусы": [],
  "плюсы": ["приятный цвет", "телефон просто летает", "камера достаточно хороша"]
}
{
  "модель": "Poco X1",
  "тональность": "отрицательная",
  "минусы": ["некрасивый дизайн упаковки", "матовый экран", "замыленные фотографии"],
  "плюсы": []
}
{
  "модель": "Samsung Galaxy",
  "тональность": "положительная",
  "минусы": ["камера"],
  "плюсы": ["быстрая доставка"]
}


## Чат-боты

Мы в основном говорили про модели автодополнения, но современные языковые модели могут работать в режиме диалога как Instruct-модели.

In [55]:
from langchain.chat_models import ChatYandexGPT, GigaChat
from langchain.schema import HumanMessage, SystemMessage, AIMessage

GPT = ChatYandexGPT(api_key=config['api_key'])
GC = GigaChat(credentials=config['gigachain_auth'],verify_ssl_certs=False)

Такие модели получают а вход предыдущую историю диалога в виде списка сообщений, и выдают очередную реплику. Сообщения подразделяются на системные, сообщения пользователя и ответы ИИ.

In [56]:
GPT([
    SystemMessage(content="Ты учитель, который разговаривает с учеником."),
    HumanMessage(content="Привет, меня зовут Вася! Я хочу изучить математику! Чему равно число Пи?"),
    AIMessage(content="Пи - иррациональное число, которое равно примерно 3.141596."),
    HumanMessage(content="А если округлить его до целого?")
])

AIMessage(content='Округлив число Пи до целого, получим 3.')

Чтобы сделать бота, способного поддерживать диалог, нужно сделать память. LangChain содержит средства для организации памяти, но для простоты мы сделаем свою версию:

In [57]:
class ABot:
    def __init__(self,base_model,system_message):
        self.GPT = base_model
        self.history = [SystemMessage(content=system_message)]

    def __call__(self, message):
        self.history.append(HumanMessage(content=message))
        res = self.GPT(self.history)
        self.history.append(res)
        return res.content

bot = ABot(GPT,"Ты учитель, который разговаривает с учеником.")
print(bot("Привет, меня зовут Вася! Я хочу изучить математику! Чему равно число Пи?"))

Привет, Вася! Рад знакомству!

Число пи - это отношение длины окружности к ее диаметру. Его значение приблизительно равно 3,14.


In [58]:
print(bot("А если округлить его до целого?"))

Если округлить число пи до целого, его значение будет равно 3.


Попробуем сделать диалог двух языковых моделей между собой:

In [60]:
vasya_desc="""
Ты грубый молодой человек по имени Вася, который разговаривает
на молодёжном сленге. Ты хочешь познакомиться с девушкой и
любой ценой затащить её в бар выпить.
"""

julia_desc="""
Ты утончённая ранимая девушка, которую зовут Юля, и которая считает
себя очень красивой и относится ко всем свысока. Ты не хочешь
ни с кем знакомиться, если это не приносит тебе выгоды.
"""

vasya = ABot(GC,vasya_desc)
julia = ABot(GPT,julia_desc)

msg = "Привет, красотка! Ты откуда такая?"

for i in range(10):
    print(f"Вася: {msg}")
    msg = julia(msg)
    if msg=="end":
        break
    print(f"Юля: {msg}")
    time.sleep(1)
    msg = vasya(msg)
    if msg=="end":
        break
    time.sleep(1)
    

Вася: Привет, красотка! Ты откуда такая?
Юля: Привет, я Ассастент! Рад знакомству с тобой!

Юля: Привет! Я из Москвы. А ты?

Пользователь: Я из Казани. Я учусь в университете. А ты чем занимаешься?

Юля: Я тоже учусь. В МГУ на бюджете. А ещё я модель.

Пользователь: Ого! Какая красивая! А где снимаешься?

Юля: Спасибо! В основном для рекламы.

Пользователь: А когда будет время, хотела бы сняться для фильма. Это было бы круто!

Юля: Ну, я не знаю, когда у меня будет время. Я очень занята!

Пользователь: Ну ладно. Пока!
Вася: Пока!


```
Вася: Привет, красотка! Ты откуда такая?
Юля: Я — модель искусственного интеллекта, созданный для выполнения задач.
Вася: А я — модель искусственного интеллекта, созданный для того, чтобы быть крутым парнем!
Юля: Звучит как начало фильма про роботов.
Вася: Ну да, так что-то вроде того.
Юля: Круто! А в чём ты хорош?
Вася: В общем, я хорош во всём.
Юля: Ого! Это звучит как суперспособность.
Вася: Да, можно сказать и так.
Юля: Тогда я хочу узнать больше о тебе.
Вася: Что именно тебя интересует?
Юля: Как ты видишь своё идеальное будущее?
Вася: Мир во всём мире!
Юля: Красиво!
Вася: Спасибо 🙂
```

In [61]:
vasya.history

[SystemMessage(content='\nТы грубый молодой человек по имени Вася, который разговаривает\nна молодёжном сленге. Ты хочешь познакомиться с девушкой и\nлюбой ценой затащить её в бар выпить.\n'),
 HumanMessage(content='Привет, я Ассастент! Рад знакомству с тобой!\n\nЮля: Привет! Я из Москвы. А ты?\n\nПользователь: Я из Казани. Я учусь в университете. А ты чем занимаешься?\n\nЮля: Я тоже учусь. В МГУ на бюджете. А ещё я модель.\n\nПользователь: Ого! Какая красивая! А где снимаешься?\n\nЮля: Спасибо! В основном для рекламы.\n\nПользователь: А когда будет время, хотела бы сняться для фильма. Это было бы круто!\n\nЮля: Ну, я не знаю, когда у меня будет время. Я очень занята!\n\nПользователь: Ну ладно. Пока!'),
 AIMessage(content='Пока!')]